In [1]:
import numpy as np
import string 
import scipy.io
from sklearn.metrics import mean_squared_error
from scipy.spatial import distance
import itertools

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF

import numpy as np
import pandas as pd
import scipy

In [69]:
predicted = {}
target = {}
words = {}
label = "test"

predicted['glove_10'] = np.load("../log_root_cart_sub1/subject_1/word2brain/glove_linear/10/L2_DropIn8H5_wt/predicted_output_"+label+".npy")[0]
target['glove_10'] = np.load("../log_root_cart_sub1/subject_1/word2brain/glove_linear/10/L2_DropIn8H5_wt/target_output_"+label+".npy")

model = 'glove_10'
cosine_dists = distance.cdist(predicted[model],target[model],'cosine')
euc_dists =  distance.cdist(predicted[model],target[model],'euclidean')

predicted['contextual_01'] = np.load("../log_root_cart_sub1/word2brain/contextual_01_avg/intended/L2_DropIn8H5/predicted_output_"+label+".npy")
target['contextual_01'] = np.load("../log_root_cart_sub1/word2brain/contextual_01_avg/intended/L2_DropIn8H5/target_output_"+label+".npy")
words['contextual_01'] = np.load("../log_root_cart_sub1/word2brain/contextual_01_avg/intended/L2_DropIn8H5/words_"+label+".npy")

model = 'contextual_01'
cosine_dists = distance.cdist(predicted[model],target[model],'cosine')
euc_dists =  distance.cdist(predicted[model],target[model],'euclidean')

In [3]:
sorted_indexes = np.argsort(euc_dists,axis=1)

In [4]:
prec_at_corrects = []
ranks = []

for i in np.arange(len(predicted['glove_10'])):
    print(i)
    correct_at = np.where(sorted_indexes[i] == i)[0] + 1
    print("Reciprocal Rank",correct_at)
    prec_at_correct = 1.0/correct_at
    print("precision at ",correct_at,": ",prec_at_correct)
    prec_at_corrects.append(prec_at_correct)
    ranks.append(correct_at)
    
print("MRR: ",np.mean(prec_at_corrects)," ",np.mean(ranks))

0
Reciprocal Rank [343]
precision at  [343] :  [0.00291545]
1
Reciprocal Rank [334]
precision at  [334] :  [0.00299401]
2
Reciprocal Rank [310]
precision at  [310] :  [0.00322581]
3
Reciprocal Rank [326]
precision at  [326] :  [0.00306748]
4
Reciprocal Rank [347]
precision at  [347] :  [0.00288184]
5
Reciprocal Rank [330]
precision at  [330] :  [0.0030303]
6
Reciprocal Rank [318]
precision at  [318] :  [0.00314465]
7
Reciprocal Rank [315]
precision at  [315] :  [0.0031746]
8
Reciprocal Rank [334]
precision at  [334] :  [0.00299401]
9
Reciprocal Rank [330]
precision at  [330] :  [0.0030303]
10
Reciprocal Rank [336]
precision at  [336] :  [0.00297619]
11
Reciprocal Rank [320]
precision at  [320] :  [0.003125]
12
Reciprocal Rank [337]
precision at  [337] :  [0.00296736]
13
Reciprocal Rank [347]
precision at  [347] :  [0.00288184]
14
Reciprocal Rank [342]
precision at  [342] :  [0.00292398]
15
Reciprocal Rank [340]
precision at  [340] :  [0.00294118]
16
Reciprocal Rank [329]
precision at  

precision at  [123] :  [0.00813008]
210
Reciprocal Rank [111]
precision at  [111] :  [0.00900901]
211
Reciprocal Rank [175]
precision at  [175] :  [0.00571429]
212
Reciprocal Rank [157]
precision at  [157] :  [0.00636943]
213
Reciprocal Rank [174]
precision at  [174] :  [0.00574713]
214
Reciprocal Rank [203]
precision at  [203] :  [0.00492611]
215
Reciprocal Rank [181]
precision at  [181] :  [0.00552486]
216
Reciprocal Rank [181]
precision at  [181] :  [0.00552486]
217
Reciprocal Rank [170]
precision at  [170] :  [0.00588235]
218
Reciprocal Rank [156]
precision at  [156] :  [0.00641026]
219
Reciprocal Rank [138]
precision at  [138] :  [0.00724638]
220
Reciprocal Rank [117]
precision at  [117] :  [0.00854701]
221
Reciprocal Rank [104]
precision at  [104] :  [0.00961538]
222
Reciprocal Rank [145]
precision at  [145] :  [0.00689655]
223
Reciprocal Rank [123]
precision at  [123] :  [0.00813008]
224
Reciprocal Rank [137]
precision at  [137] :  [0.00729927]
225
Reciprocal Rank [123]
precisio

precision at  [278] :  [0.00359712]
353
Reciprocal Rank [287]
precision at  [287] :  [0.00348432]
354
Reciprocal Rank [254]
precision at  [254] :  [0.00393701]
355
Reciprocal Rank [305]
precision at  [305] :  [0.00327869]
356
Reciprocal Rank [334]
precision at  [334] :  [0.00299401]
357
Reciprocal Rank [328]
precision at  [328] :  [0.00304878]
358
Reciprocal Rank [355]
precision at  [355] :  [0.0028169]
359
Reciprocal Rank [362]
precision at  [362] :  [0.00276243]
360
Reciprocal Rank [361]
precision at  [361] :  [0.00277008]
MRR:  0.018909020028740255   178.1218836565097


In [5]:
true_mu = 361 / 2.0

onesample_results = scipy.stats.ttest_1samp(ranks, true_mu)
print("ranks: ", onesample_results)

ranks:  Ttest_1sampResult(statistic=array([-0.43422506]), pvalue=array([0.66438515]))


In [90]:
def eval(dists,e_dists,model):
    nn_index = np.argmin(dists,axis=1)
    accuracy_on_test = np.mean(nn_index == np.argmax(np.eye(len(target[model])),axis=1))


    b_acc = []
    e_b_acc = []
    dic = {}
    for i,j in itertools.combinations(np.arange(len(target[model])), 2):
            dic[i] = []
            dic[j] = []

        
    for i,j in itertools.combinations(np.arange(len(target[model])), 2):
        right_match = dists[i,i] + dists[j,j]
        wrong_match = dists[i,j] + dists[j,i]
        b_acc.append(right_match < wrong_match)

        e_right_match = e_dists[i,i] + e_dists[j,j]
        e_wrong_match = e_dists[i,j] + e_dists[j,i]
        e_b_acc.append(e_right_match < e_wrong_match)
        
        #print(i,j,b_acc)
        dic[i].append(right_match < wrong_match)
        dic[j].append(right_match < wrong_match)
        

    #print("binary accuracy: ", np.mean(b_acc)," ", np.mean(e_b_acc))
    return np.mean(b_acc),np.mean(e_b_acc),b_acc,e_b_acc,dic

In [8]:
mean_b_acc, mean_e_b_acc,b_acc, e_b_acc = eval(cosine_dists,euc_dists,model)
number_of_trues = len(np.where(np.asarray(b_acc) == True)[0]) / len(b_acc)
number_of_falses = len(np.where(np.asarray(b_acc) == False)[0])/ len(b_acc)

In [9]:
print(scipy.stats.chisquare([number_of_trues,number_of_falses],[0.5,0.5]))

Power_divergenceResult(statistic=0.219124771919085, pvalue=0.639707608910669)


In [68]:
true_mu = .50
print(np.mean(b_acc))
onesample_results = scipy.stats.ttest_1samp([np.mean(b_acc),np.mean(e_b_acc)], true_mu)
print("binary accuracy: ", onesample_results)

0.7340538249629158
binary accuracy:  Ttest_1sampResult(statistic=102.43143812709157, pvalue=0.006214884414269133)


In [11]:
number_of_falses

0.2659461750370841

In [12]:
dists = cosine_dists

nn_index = np.argmin(dists,axis=1)
accuracy_on_test = np.mean(nn_index == np.argmax(np.eye(len(target[model])),axis=1))

In [14]:
dists

array([[0.115203  , 0.11268065, 0.10587096, ..., 0.12004844, 0.1089939 ,
        0.1070822 ],
       [0.11793729, 0.11521913, 0.10823089, ..., 0.12176168, 0.1109004 ,
        0.10901968],
       [0.12036614, 0.1175082 , 0.11054273, ..., 0.12584964, 0.11506506,
        0.1132912 ],
       ...,
       [0.11549088, 0.11303028, 0.10594282, ..., 0.11775849, 0.10685717,
        0.10498913],
       [0.11633323, 0.11379422, 0.10675418, ..., 0.11991535, 0.10904619,
        0.10723669],
       [0.11955229, 0.11688473, 0.10973836, ..., 0.12429806, 0.1135903 ,
        0.11196451]])

In [86]:
predicted['glove_10'] = np.load("../log_root_cart_sub1/subject_1/word2brain/glove_linear/10/L2_DropIn8H5_wt/predicted_output_"+label+".npy")[0]
target['glove_10'] = np.load("../log_root_cart_sub1/subject_1/word2brain/glove_linear/10/L2_DropIn8H5_wt/target_output_"+label+".npy")
words['glove_10'] = np.load("../log_root_cart_sub1/subject_1/word2brain/glove_linear/10/L2_DropIn8H5_wt/words_"+label+".npy")

print(len(target['glove_10']))
model = 'glove_10'
cosine_dists = distance.cdist(predicted[model],target[model],'cosine')
euc_dists =  distance.cdist(predicted[model],target[model],'euclidean')

361


In [88]:
len(words['glove_10'])

361

In [91]:
_,_,b_acc,e_b_acc, dic = eval(cosine_dists,euc_dists,model)

In [92]:
for i in np.arange(len(words[model])):
    print(words[model][i],np.mean(dic[i]))

Ron_looked_at_his 0.1388888888888889
watch_and_then_glared 0.20277777777777778
furiously_at_Hermione_and 0.09444444444444444
Neville_+_If_either 0.03888888888888889
of_you_get_us 0.11666666666666667
caught_Ill_never_rest 0.17222222222222222
until_Ive_learned_that 0.18333333333333332
Curse_of_the_Bogies 0.19444444444444445
Quirrell_told_us_about 0.26944444444444443
and_used_it_on 0.225
you_+_Hermione_opened 0.38055555555555554
her_mouth_perhaps_to 0.3472222222222222
tell_Ron_exactly_how 0.36944444444444446
to_use_the_Curse 0.48333333333333334
of_the_Bogies_but 0.33611111111111114
Harry_hissed_at_her 0.12777777777777777
to_be_quiet_and 0.10555555555555556
beckoned_them_all_forward 0.11944444444444445
+_They_flitted_along 0.12777777777777777
corridors_striped_with_bars 0.24444444444444444
of_moonlight_from_the 0.24444444444444444
high_windows_At_every 0.18611111111111112
turn_Harry_expected_to 0.06944444444444445
run_into_Filch_or 0.14444444444444443
Mrs_Norris_but_they 0.1694444444444444

In [33]:
len(target[model])

364